In [2]:
# !wget https://huggingface.co/datasets/mesolitica/fb-malaysian-pages/resolve/main/dedup.jsonl

In [3]:
import openai

openai_key = ''
openai.api_key = openai_key

In [4]:
# response = openai.Embedding.create(
#         input=t,
#         model="text-embedding-ada-002"
#     )
#     embeddings = response['data'][0]['embedding']

In [5]:
import os
import json

texts = []
with open('dedup.jsonl') as fopen:
    for l in fopen:
        texts.append(json.loads(l))

In [6]:
len(texts)

193375

In [7]:
texts[:10]

['Adeyyy pooodahhh puiii',
 'Bosan la ceramah dia.. dulu sonok gak',
 'Memalukan betul.. PM malaysia.\nDunia sedang memerhatikan Malaysia dipimpin oleh orang yang tercemar dgn kes2 mahkamah... \n\nMalaysia PM Picks Graft-Tainted Leader as One of His Deputies https://\nwww.bloomberg.co\nm/news/\narticles/\n2022-12-02/\nmalaysia-pm-pick\ns-graft-tainted\n-leader-as-one-\nof-his-deputies',
 'Tahniah RR....\nInilah yg rakyat harapakan, jwtn Menteri Ekonomi demi pertiwi...',
 'doakan mb kau ni jaga negeri elok2',
 ' VS \n yng mati kena pijak.',
 'smoga trus dirahmati',
 'Dalam gov unity Anwar, D4P ada 4 menteri? Eh bukan 3A menteri je ke?',
 'Mahanom Mohammad habis la modal dorg puan.. xjd lah buat prn kedah tganu klntn gini .hihi',
 'Abdul Azim Che Hassan laaaaa org2 Pas boleh la xkan xtahu...']

In [8]:
import re
from tqdm import tqdm

def cleaning(string):
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('@') < 0]
        ),
    )
    return string

cleaned_texts = [(cleaning(t), t) for t in tqdm(texts)]
cleaned_texts = [t for t in cleaned_texts if len(t[0]) > 5]

100%|██████████| 193375/193375 [00:00<00:00, 353585.05it/s]


In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [10]:
directory = 'facebook'
!mkdir {directory}

In [11]:
def translate(no, item):
    filename = os.path.join(directory, f'{no}.json')
    if os.path.exists(filename):
        return True
    
    r = None
    try:
        o = openai.Embedding.create(
            input=item[0],
            model="text-embedding-ada-002"
        )
        r = {
            'v': o['data'][0]['embedding'],
            'text': item,
        }
    except:
        pass
    fopen = open(filename, 'w')
    if r is not None:
        json.dump(r, fopen)
    fopen.close()
    return True

In [13]:
cleaned_texts[1000]

('Freddz Othman K edah K alah K oyak', 'Freddz Othman K edah K alah K oyak')

In [14]:
%%time
translate(10000, cleaned_texts[10000])

CPU times: user 50.1 ms, sys: 0 ns, total: 50.1 ms
Wall time: 389 ms


True

In [15]:
CHUNK_SIZE = 50
start = 0
for i in tqdm(range(start, len(cleaned_texts), CHUNK_SIZE)):
    b = [(i + k, cleaned_texts[i + k]) for k in range(len(cleaned_texts[i: i + CHUNK_SIZE]))]

    with ThreadPoolExecutor(max_workers=len(b)) as executor:
        futures = {executor.submit(translate, item[0], item[1]): item for item in b}

        for future in as_completed(futures):
            future.result()

100%|██████████| 3859/3859 [10:54:04<00:00, 10.17s/it]  
